In [1]:
import torch.nn.functional as F
from cleantext import clean

import torch
import pandas
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/remi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
queries = read_excel("/home/remi/Downloads/Export.recherches.site.du.01-01-23.au.31-08-23.xlsx")
export_recherches = queries["Nom"].apply(lambda q : "query:"+str(q)).to_list()

questions_raw = read_excel("/home/remi/Downloads/questions-fulltext.xlsx", header=None).dropna()
questions = questions_raw[0].apply(lambda q : "query:"+clean(str(q), to_ascii=False)).to_list()

fiches_mt = read_json("/home/remi/Downloads/fiches-min-travail.json")
#passages = fiches_mt["text_full"].apply(lambda q : "query:"+str(q)).to_list()


#full_data = passages + questions

In [3]:
# embbed each individually
def run_embeddings(elements):
    p_emb = []
    i = 0
    for p in elements :
        i+=1
        print(i)
        #print(sys.getsizeof(p_emb))
        batch_dict = tokenizer(p, max_length=512, padding=True, truncation=True, return_tensors='pt')
    
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
        #print(normalized_embeddings)
        p_emb.append(normalized_embeddings.detach())
    return torch.squeeze(torch.stack(p_emb))
        #time.sleep(1)

In [4]:
question_elements = [(lambda q : "query:" + q)(q) for q in questions]
tensor_questions = run_embeddings(question_elements)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [64]:
tensor_questions.size()

torch.Size([51, 768])

In [5]:
def treat_fiche(fiche_title, text_full, text_by_section):
    if (len(text_by_section)==0):
        text_by_section = [{'text': text_full, 'title': ''}]
    return [(lambda t : [fiche_title, clean(fiche_title + ' ' + t['title'] + ' ' + t['text'], to_ascii=False)])(t) for t in text_by_section]


In [6]:
treated = fiches_mt.apply(lambda f: treat_fiche(f.title, f.text_full, f.text_by_section), axis=1)

In [7]:
stacked = treated.apply(pandas.Series).stack().reset_index(drop=True)

In [8]:
passages_set = pandas.DataFrame(stacked.values.tolist())

In [9]:
#passages_elements = passages_set.applymap(lambda a : 'passage: ' + a).to_numpy().tolist()
#passages_elements[0]
passages_set[1].apply(lambda p : [p])
tensor_docs = run_embeddings(passages_set[1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
tensor_docs.size()

torch.Size([1304, 768])

In [21]:
tensor_docs.numpy().shape

(1304, 768)

In [12]:
scores = (tensor_questions @ tensor_docs.T) * 100

In [14]:
for i in range(0, len(questions)):
#for i in range(0, 3):
    retrieval = 15
    j = torch.argmax(scores[i]).item()
    matches = torch.topk(scores[i], retrieval).values.tolist()
    matching_indices = torch.topk(scores[i], retrieval).indices.tolist()
    print(matches)
    print(matching_indices)

    # reranker
    batch = [questions[i]]

    for m in matching_indices :    
        batch.append('passage: ' + passages_set.loc[m][1])
    #print(batch)
    batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    normalized_embeddings = F.normalize(embeddings, p=2, dim=1).detach()

    reranker_scores = (normalized_embeddings[0] @ normalized_embeddings[1:].T) * 100
    print(reranker_scores)
    print(reranker_scores.size())

    rr_matches = torch.topk(reranker_scores, 3).values.tolist()
    rr_matching_indices = torch.topk(reranker_scores, 3).indices.tolist()
    print(len(rr_matching_indices))
    print(rr_matching_indices)
    print(questions[i])
    
    for rrm in rr_matching_indices :
        # -1 since first is query itself
        m = matching_indices[rrm]
        print(m)
        print("///////////////////////////////////////////////////////")
        print(passages_set.loc[m][0])
        print(passages_set.loc[m][1])
        print()
        
    print()

[89.37093353271484, 89.31256866455078, 88.45869445800781, 88.40901947021484, 87.73321533203125, 87.545654296875, 87.39739990234375, 87.38876342773438, 87.10906219482422, 86.64740753173828, 86.63619232177734, 86.61006927490234, 86.58133697509766, 86.50524139404297, 86.44860076904297]
[440, 469, 447, 533, 449, 472, 441, 479, 413, 414, 45, 438, 470, 426, 296]
tensor([86.8199, 85.5012, 84.8943, 85.7715, 84.4480, 85.6086, 85.8036, 85.0924,
        84.9328, 84.7642, 84.8892, 84.5117, 84.6998, 84.7047, 83.4055])
torch.Size([15])
3
[0, 6, 3]
query:je suis tombé malade pendant mes congés, est-ce que mes congés peuvent être repoussés ?
440
///////////////////////////////////////////////////////
Les congés payés
quelle est l'incidence de l'absence pour maladie sur le droit aux congés payés ? les absences pour maladie n'ouvrent pas droit à des congés payés, sauf dispositions conventionnelles contraires.en revanche, l'employeur ne saurait déduire du congé annuel les jours d'absence pour maladie.

4

In [50]:
fiches_mt["text_by_section"][0]

[{'title': '1) Visite d’information et de prévention\xa0: pour qui\xa0?',
  'text': 'Depuis le 1er janvier 2017, la visite d’information et de prévention vient remplacer la traditionnelle visite médicale d’embauche qui devait être réalisée au plus tard avant la fin de la période d’essai.Désormais, la visite doit être organisée dans un délai maximum de trois mois à compter de l’occupation effective du poste de travail (sauf cas de dispense). Néanmoins, pour certains salariés, comme les jeunes de moins de 18 ans ou  les travailleurs de nuit, elle doit toujours avoir lieu avant la prise de fonction.Bon à savoir\xa0! Tout salarié affecté à un poste présentant des risques particuliers pour sa santé (exposition à l’amiante, au plomb, à des agents cancérogènes, etc.) doit passer, avant sa prise de fonction, un examen médical d’aptitude et non une simple visite d’information et de prévention.La visite d’information et de prévention permet d’interroger le salarié sur son état de santé, et d’env

In [60]:
fiches_mt["text_by_section"][0][1]['text']

'Le Code du travail fixe à 5 ans le délai maximum entre deux visites d’information et de prévention à la médecine du travail. Mais cet intervalle est ramené à trois ans pour les salariés dont l’état de santé, l’âge, les conditions de travail ou les risques professionnels auxquels ils sont exposés le nécessitent. Sont, par exemple, concernés par ce suivi adapté\xa0: les travailleurs handicapés, les salariés titulaires d’une pension d’invalidité et les travailleurs de nuit.Bon à savoir\xa0! Les salariés affectés à des postes à risques bénéficient d’un suivi renforcé. Le délai entre deux examens par le médecin du travail ne peut pas excéder quatre ans. Et, une visite intermédiaire est effectuée par un professionnel de santé au plus tard deux ans après la visite avec le médecin du travail.Pour davantage de détails consultez notre fiche sur le suivi de l’état de santé des salariés.'

In [63]:
from cleantext import clean
ct = clean(fiches_mt["text_by_section"][0][1]['text'])

In [29]:
def treat_fiche(fiche_title, text_full, text_by_section):
    if (len(text_by_section)==0):
        text_by_section = [{'text': text_full, 'title': ''}]
    return [(lambda t : [fiche_title, clean(t['title'] + ' ' + t['text'])])(t) for t in text_by_section]


In [30]:
treated = fiches_mt.apply(lambda f: treat_fiche(f.title, f.text_full, f.text_by_section), axis=1)

In [15]:
stacked = treated.apply(pandas.Series).stack().reset_index(drop=True)

In [33]:
passages_set.loc[10]

0     5 questions-réponses sur le handicap et l’emploi
1    question 1: comment faire reconnaitre son hand...
Name: 10, dtype: object

In [12]:
def search(query):
    # embbed query
    batch_dict = tokenizer('query: '+ query, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    question_normalized_embeddings = F.normalize(embeddings, p=2, dim=1).detach()

    # score against all passages and retrieve top k
    
    retrieval_scores = (question_normalized_embeddings @ tensor_docs.T)[0] * 100
    k = 15
    
    j = torch.argmax(retrieval_scores).item()
    matches = torch.topk(retrieval_scores, k).values.tolist()
    matching_indices = torch.topk(retrieval_scores, k).indices.tolist()
    print(matches)
    print(matching_indices)

    # reranker
    batch = ['query: '+ query]

    for m in matching_indices :    
        batch.append('passage: ' + passages_set.loc[m][1])
        #print(batch)
        
    batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    normalized_embeddings = F.normalize(embeddings, p=2, dim=1).detach()

    reranker_scores = (normalized_embeddings[0] @ normalized_embeddings[1:].T) * 100
    print(reranker_scores)
    print(reranker_scores.size())
    
    n = 3
    rr_matches = torch.topk(reranker_scores, n).values.tolist()
    rr_matching_indices = torch.topk(reranker_scores, n).indices.tolist()
    # print(len(rr_matching_indices))
    # print(rr_matching_indices)
    # print(questions[i])
    
    for rrm in rr_matching_indices :
        # -1 since first is query itself
        m = matching_indices[rrm]
        print(m)
        print("///////////////////////////////////////////////////////")
        print(passages_set.loc[m][0])
        print(passages_set.loc[m][1])
        print()
        
    # print()
    

In [22]:
search("apprentissage salaire")

[85.6441650390625, 85.04998016357422, 84.34626007080078, 83.93567657470703, 83.79011535644531, 83.67963409423828, 83.54559326171875, 83.3520278930664, 83.28624725341797, 83.03084564208984, 82.89350128173828, 82.76893615722656, 82.67122650146484, 82.33065795898438, 82.25288391113281]
[789, 26, 27, 25, 786, 788, 787, 28, 302, 18, 29, 790, 7, 8, 759]
tensor([85.5850, 84.8884, 84.6829, 84.0993, 83.7164, 83.3821, 83.5264, 83.6063,
        83.1773, 83.0891, 82.7804, 82.7057, 82.5304, 82.5428, 82.0847])
torch.Size([15])
789
///////////////////////////////////////////////////////
Le contrat d’apprentissage aménagé
le contrat d'apprentissage aménagé quelle est la rémunération ? comme tout apprenti, le jeune salarié handicapé est rémunéré en pourcentage du smic, variable selon son âge et sa progression dans le ou les cycles de formation faisant l'objet de l'apprentissage.année d'exécution du contratapprenti de moins de 18 ansapprenti de 18 ans à moins de 21 ansapprenti de 21 ans et plus
1ère ann